In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import accessDB as db
from scipy import stats

pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.6.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.4
pytest: 3.8.2
pip: 19.0.3
setuptools: 41.0.1
Cython: 0.28.5
numpy: 1.16.3
scipy: 1.1.0
pyarrow: None
xarray: None
IPython: 7.0.1
sphinx: 1.8.1
patsy: 0.5.0
dateutil: 2.7.3
pytz: 2018.5
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.8
feather: None
matplotlib: 2.2.2
openpyxl: 2.5.8
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.1.1
lxml: 4.2.5
bs4: 4.6.3
html5lib: 0.9999999
sqlalchemy: 1.2.11
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: 0.7.0


In [2]:
# Read raw data in
df = db.filter(doGeoLoc=True)

In [3]:
df.head()

,Call Number,Date/Time,Location,Police District,Nature of Call,Status,Latitude,Longitude
ID,,,,,,,,
182551107-Unable to Locate Complainant,182551107,2018-09-12 11:51:11,"S 13TH ST / W MORGAN AV,MKE",6,ACC PDO,Unable to Locate Complainant,42.9811,-87.9289
182551111-Assignment Completed,182551111,2018-09-12 11:51:45,"500 E OAK ST, OAK CREEK",4,PROPERTY PICKUP,Assignment Completed,NaN,NaN
182551115-Assignment Completed,182551115,2018-09-12 11:53:49,"2333 N 49TH ST,MKE",3,RETURN STATION,Assignment Completed,43.0614,-87.9755
182551113-Advised/Referral,182551113,2018-09-12 11:54:07,"5348 N 91ST ST,MKE",4,ENTRY,Advised/Referral,43.1151,-88.0253
182551120-Service in Progress,182551120,2018-09-12 11:54:36,"6929 W SILVER SPRING DR,MKE",4,OUT OF SERVICE,Service in Progress,43.1189,-87.9982


In [4]:
district_status = df[['Status','Police District']].set_index('Police District')
district_status.head()

,Status
Police District,
6,Unable to Locate Complainant
4,Assignment Completed
3,Assignment Completed
4,Advised/Referral
4,Service in Progress


In [17]:
unique_statuses = district_status.Status.value_counts().index[:14].tolist()
unique_statuses

['Service in Progress',
 'Assignment Completed',
 'Advised',
 'Unable to Locate Complainant',
 'To be Filed',
 'City Citation(s) Issued',
 'Advised/Referral',
 'Open Investigation',
 'No Prosecution',
 'Cleared by Arrest',
 'Filed Driver Exchange Report',
 'False Alarm',
 'Referral',
 'Patrol Request']

In [18]:
district_status.Status.value_counts()

Service in Progress              157075
Assignment Completed             147855
Advised                           65488
Unable to Locate Complainant      31517
To be Filed                       16187
City Citation(s) Issued           12972
Advised/Referral                   8440
Open Investigation                 4192
No Prosecution                      506
Cleared by Arrest                   479
Filed Driver Exchange Report        333
False Alarm                         325
Referral                            116
Patrol Request                       80
False Alarm (Weather Related)        13
State Citation(s) Issued             10
Ordered to Appear                     1
Name: Status, dtype: int64

In [7]:
districts = ['1','2','3','4','5','6','7']
status_freq = pd.crosstab(df['Police District'],df['Status'])
status_freq = status_freq[status_freq.index.isin(districts)]
status_freq = status_freq[unique_statuses]
status_freq

Status,Unable to Locate Complainant,Assignment Completed,Advised/Referral,Service in Progress,City Citation(s) Issued,Advised,To be Filed,Open Investigation,Filed Driver Exchange Report,Cleared by Arrest,No Prosecution,Referral,Patrol Request,False Alarm
Police District,,,,,,,,,,,,,,
1,1615,16887,436,18766,1466,6465,2100,408,54,34,54,13,4,44
2,4145,21222,1211,20088,1578,7888,2341,494,62,86,54,22,13,36
3,6168,25413,1294,29004,1463,9016,2304,777,37,110,64,19,8,75
4,4517,21329,1303,23158,1964,7665,1974,609,54,56,82,10,15,53
5,6209,18442,1861,22904,2209,12651,2343,552,22,58,100,21,10,26
6,2512,19431,626,15066,1583,7943,2088,476,64,45,54,10,16,40
7,6259,24133,1703,26851,2552,13608,2930,866,39,87,98,20,14,51


In [8]:
total_statuses = status_freq.sum(axis=1).tolist()
percents = pd.DataFrame()

for index, total in enumerate(total_statuses):
    i = str(index+1)
    row = status_freq[status_freq.index == i]\
        .apply(lambda x: x/total_statuses[index],axis=0)
    percents = pd.concat([percents, row])

percents

Status,Unable to Locate Complainant,Assignment Completed,Advised/Referral,Service in Progress,City Citation(s) Issued,Advised,To be Filed,Open Investigation,Filed Driver Exchange Report,Cleared by Arrest,No Prosecution,Referral,Patrol Request,False Alarm
Police District,,,,,,,,,,,,,,
1,0.033405,0.349295,0.009018,0.388160,0.030323,0.133724,0.043437,0.008439,0.001117,0.000703,0.001117,0.000269,0.000083,0.000910
2,0.069970,0.358238,0.020442,0.339095,0.026637,0.133153,0.039517,0.008339,0.001047,0.001452,0.000912,0.000371,0.000219,0.000608
3,0.081424,0.335476,0.017082,0.382881,0.019313,0.119020,0.030415,0.010257,0.000488,0.001452,0.000845,0.000251,0.000106,0.000990
4,0.071939,0.339693,0.020752,0.368823,0.031279,0.122076,0.031439,0.009699,0.000860,0.000892,0.001306,0.000159,0.000239,0.000844
5,0.092111,0.273588,0.027608,0.339782,0.032771,0.187678,0.034758,0.008189,0.000326,0.000860,0.001484,0.000312,0.000148,0.000386
6,0.050286,0.388978,0.012532,0.301597,0.031689,0.159006,0.041798,0.009529,0.001281,0.000901,0.001081,0.000200,0.000320,0.000801
7,0.079017,0.304667,0.021500,0.338981,0.032218,0.171794,0.036990,0.010933,0.000492,0.001098,0.001237,0.000252,0.000177,0.000644


In [9]:
districts = ['1','2','3','4','5','6','7']
natures = list(df['Nature of Call'].value_counts()[:14].index)
nature_freq = pd.crosstab(df['Police District'],df['Nature of Call'])
nature_freq = nature_freq[nature_freq.index.isin(districts)]
nature_freq = nature_freq[natures]
nature_freq

Nature of Call,BUSINESS CHECK,TRAFFIC STOP,TRBL W/SUBJ,RETURN STATION,FOLLOW UP,PATROL,WELFARE CITIZEN,REPORTS,ACC PDO,INVESTIGATION,POLICE ADMIN,SPECIAL ASSIGN,SUSP PERS/AUTO,CALL FOR POLICE
Police District,,,,,,,,,,,,,,
1,6662,4424,3508,2208,1887,1478,1308,1328,1620,830,1886,1879,676,380
2,6098,4089,3494,3312,2319,2997,1567,1819,1962,1318,929,1097,1077,938
3,5411,4916,4194,5029,2975,1946,2016,2308,1943,3385,3636,2061,1227,1787
4,4677,4398,3272,3936,2160,4697,2106,1708,1570,1798,1551,1620,1357,1603
5,5201,8690,4088,2954,2170,1063,1943,2011,1434,2318,1039,1075,1013,1683
6,6849,4758,2711,2951,2068,3309,1751,1831,1690,629,1101,726,1562,515
7,9569,9911,4256,4488,2582,581,2274,1830,2053,1474,1296,836,1967,1671


In [10]:
total_natures = nature_freq.sum(axis=1).tolist()
percents = pd.DataFrame()

for index, total in enumerate(total_natures):
    i = str(index+1)
    row = nature_freq[nature_freq.index == i]\
        .apply(lambda x: x/total_natures[index],axis=0)
    percents = pd.concat([percents, row])

percents

Nature of Call,BUSINESS CHECK,TRAFFIC STOP,TRBL W/SUBJ,RETURN STATION,FOLLOW UP,PATROL,WELFARE CITIZEN,REPORTS,ACC PDO,INVESTIGATION,POLICE ADMIN,SPECIAL ASSIGN,SUSP PERS/AUTO,CALL FOR POLICE
Police District,,,,,,,,,,,,,,
1,0.221520,0.147104,0.116646,0.073419,0.062745,0.049145,0.043493,0.044158,0.053867,0.027599,0.062712,0.062479,0.022478,0.012635
2,0.184698,0.123849,0.105827,0.100315,0.070239,0.090774,0.047462,0.055094,0.059426,0.039920,0.028138,0.033226,0.032621,0.028410
3,0.126325,0.114769,0.097913,0.117407,0.069454,0.045431,0.047065,0.053882,0.045361,0.079026,0.084886,0.048116,0.028645,0.041719
4,0.128302,0.120649,0.089759,0.107975,0.059254,0.128851,0.057773,0.046855,0.043069,0.049324,0.042548,0.044441,0.037226,0.043974
5,0.141786,0.236901,0.111444,0.080530,0.059157,0.028979,0.052969,0.054823,0.039093,0.063192,0.028325,0.029306,0.027616,0.045881
6,0.211057,0.146621,0.083541,0.090937,0.063727,0.101969,0.053958,0.056424,0.052079,0.019383,0.033928,0.022372,0.048134,0.015870
7,0.213651,0.221287,0.095025,0.100205,0.057649,0.012972,0.050773,0.040859,0.045838,0.032911,0.028936,0.018666,0.043918,0.037309


## Types of statistical tests to apply
- One Way ANOVA – Similar to a t-test, except that this test can be used to compare  the  means  from THREE  OR  MORE groups
- Two-Way ANOVA – A very useful statistical test, because it’s the only one that  allows  you  to  compare  the  means  of TWO OR MORE groups  in response to TWO DIFFERENT INDEPENDENT VARIABLE

Considering our data, a one way anova seems the most appropriate